In [27]:
from brian2 import *
prefs.codegen.target = 'cython'

import importlib
import simulation
import utils
importlib.reload(simulation)
importlib.reload(utils)

from simulation import network
from utils import create_input, mean_rate

The goal of the model is to reproduce the effect of lowered E/I ratio on decision making. The base network used was the Wang 2002 model for probabilistic decision making. 
The real world task being simulated is a choice task between two options; the choice requires integrating evidence from a sequence of inputs. 

On a single trial, 8 pairs of bars (left and right) with differing hights are presented. Two sub-groups of neurons in the excitatory population are recorded. 1st receives input from the "left" stream, 2nd from the "right". At the end of the trial, sub-group with the higher activity corresponds to the choice of the network of the "left" or "right" option. 

In [28]:
n_trials = 1
trials = create_input(n_trials)
mu1, mu2 = mean_rate(trials[0])
print(mu1, mu2)

[25.01589668 43.8113785  13.75995119 32.13730417 31.18520905 27.94349705
 27.76538231 10.98712164] [55.37182856 12.29005446 28.23281284 46.89306141 42.22006267  4.02225717
  0.         32.01622434]


In [29]:
pop1, pop2 = network(mu1, mu2)
print(pop1)

<SpikeMonitor, recording from 'spikemonitor_2'>


In [30]:
import numpy as np
from brian2 import ms, second

def find_r(SM, dt=0.001):
    """
    Computes time dependent firing rate using the causal exponential filter;
    INPUT: SM brian2 spike monitor; dt time bin length
    OUTPUT: 
    """
    N = 240 #number of neurons in the recording
    sim_t = 5.0 #simulation time (s)
    tau = 0.02 #time constant of the filter 

    spike_times = np.asarray(SM.t / second)

    bins = np.arange(0, sim_t + dt, dt)
    counts, bin_edges = np.histogram(spike_times, bins=bins)
    counts = counts / N
    
    # generate the kernel up to 5 time constants (approx. full decay)
    t_kernel = np.arange(0, 5 * tau, dt)
    kernel = (1.0/tau) * np.exp(-t_kernel / tau)
    
    filtered_rate = np.convolve(counts, kernel, mode='full')[:len(counts)]
    
    # time vector corresponding to the calculated rates
    time_vector = bin_edges[:-1]
    
    return time_vector, filtered_rate

time_arr, rate_arr = find_r(pop1)
#plt.plot(time_arr, rate_arr)